In [1]:
pwd()

"/home/zyu20/git/BulkLMM_Analyses/BXDLongevity"

In [2]:
using CSV, DelimitedFiles, DataFrames, Missings, XLSX
using LinearAlgebra, Statistics, Optim
using Random, Distributions, LoopVectorization
# using GeneNetworkAPI, Downloads
using BenchmarkTools
using Plots

In [3]:
sampled_traits = readdlm("output/samples1000.txt", '\t') |> vec |> x -> Int.(x);

In [28]:
function meanAbsDiff(x, y)
    
    return mean(abs.(x .- y))
    
end

meanAbsDiff (generic function with 1 method)

In [29]:
function removeInfValues!(x, y)
    
    toRemove = findall(isinf.(x));
    
    for inf_val in toRemove
        
        x[inf_val[1], inf_val[2]] = 0.0;
        y[inf_val[1], inf_val[2]] = 0.0;
        
    end
    
end

removeInfValues! (generic function with 2 methods)

## Individual LP:

In [7]:
bulklmm_results_ind_null = readdlm("output/bulklmm_L_ind_null.txt", '\t');

In [8]:
bulklmm_results_ind_null_grid = readdlm("output/bulklmm_L_ind_null_grid20.txt", '\t');

In [9]:
bulklmm_results_ind_alt_grid = readdlm("output/bulklmm_L_ind_alt_grid20.txt", '\t');

In [10]:
gemma_results_ind = readdlm("output/gemma_L_ind.txt", '\t');

In [11]:
removeInfValues!(gemma_results_ind, bulklmm_results_ind_null)

In [12]:
removeInfValues!(gemma_results_ind, bulklmm_results_ind_null_grid)

In [13]:
removeInfValues!(gemma_results_ind, bulklmm_results_ind_alt_grid)

In [14]:
removeInfValues!(gemma_results_ind)

In [30]:
meanAbsDiff(bulklmm_results_ind_null, gemma_results_ind)

0.019844982572864846

In [31]:
meanAbsDiff(bulklmm_results_ind_null_grid, gemma_results_ind)

0.026090870615912806

In [32]:
meanAbsDiff(bulklmm_results_ind_alt_grid, gemma_results_ind)

0.03286788043198145

In [33]:
meanAbsDiff(bulklmm_results_ind_alt_grid, bulklmm_results_ind_null)

0.015995631848133355

In [34]:
meanAbsDiff(bulklmm_results_ind_alt_grid, bulklmm_results_ind_null_grid)

0.010085232015539014

In [35]:
findmax(abs.(bulklmm_results_ind_alt_grid .- gemma_results_ind))

(1.225190904225336, CartesianIndex(201, 52))

In [36]:
bulklmm_results_ind_alt_grid[201, 52]

13.21768068820141

In [37]:
gemma_results_ind[201, 52]

11.992489783976074

In [38]:
thrs = 3.0

3.0

In [39]:
to_reject = findall(gemma_results_ind .> 3.0);

In [40]:
to_reject_bulklmm = findall(bulklmm_results_ind_alt_grid .> 3.0);

In [41]:
to_reject

3097-element Vector{CartesianIndex{2}}:
 CartesianIndex(2334, 4)
 CartesianIndex(2335, 4)
 CartesianIndex(2336, 4)
 CartesianIndex(2337, 4)
 CartesianIndex(2338, 4)
 CartesianIndex(2339, 4)
 CartesianIndex(2340, 4)
 CartesianIndex(2341, 4)
 CartesianIndex(2342, 4)
 CartesianIndex(2343, 4)
 CartesianIndex(2344, 4)
 CartesianIndex(2345, 4)
 CartesianIndex(2346, 4)
 ⋮
 CartesianIndex(5980, 996)
 CartesianIndex(5981, 996)
 CartesianIndex(5982, 996)
 CartesianIndex(5983, 996)
 CartesianIndex(5984, 996)
 CartesianIndex(5985, 996)
 CartesianIndex(5986, 996)
 CartesianIndex(5987, 996)
 CartesianIndex(5988, 996)
 CartesianIndex(5989, 996)
 CartesianIndex(5990, 996)
 CartesianIndex(5991, 996)

In [42]:
to_reject_bulklmm

5482-element Vector{CartesianIndex{2}}:
 CartesianIndex(2907, 3)
 CartesianIndex(2908, 3)
 CartesianIndex(2909, 3)
 CartesianIndex(4280, 3)
 CartesianIndex(4281, 3)
 CartesianIndex(4282, 3)
 CartesianIndex(4283, 3)
 CartesianIndex(4284, 3)
 CartesianIndex(4285, 3)
 CartesianIndex(4286, 3)
 CartesianIndex(4287, 3)
 CartesianIndex(4288, 3)
 CartesianIndex(4289, 3)
 ⋮
 CartesianIndex(5981, 996)
 CartesianIndex(5982, 996)
 CartesianIndex(5983, 996)
 CartesianIndex(5984, 996)
 CartesianIndex(5985, 996)
 CartesianIndex(5986, 996)
 CartesianIndex(5987, 996)
 CartesianIndex(5988, 996)
 CartesianIndex(5989, 996)
 CartesianIndex(5990, 996)
 CartesianIndex(5991, 996)
 CartesianIndex(5992, 996)

In [43]:
gemma_results_ind

7321×1000 Matrix{Float64}:
 0.13297     0.00237256  0.114716    …  0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716    …  0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716    …  0.0494661    0.479688     0.0378185
 0.13297     0.00237256  0.114716       0.0494661    0.479688     0.0378185
 0.0399133   5.74931e-5  0.176414       0.323577     0.000715

In [44]:
bulklmm_results_ind_alt_grid

7321×1000 Matrix{Float64}:
 0.144266    0.00396406  0.109569    …  0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569    …  0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569    …  0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.0397299   0.00032194  0.289912       0.352193    0.00037901  0.0156391
 ⋮         

## Strain LP:

In [54]:
bulklmm_results_strains_null = readdlm("output/bulklmm_L_strains_null.txt", '\t');

In [29]:
bulklmm_results_strains_null_grid = readdlm("output/bulklmm_L_strains_null_grid20.txt", '\t');

In [30]:
bulklmm_results_strains_alt_grid = readdlm("output/bulklmm_L_strains_alt_grid20.txt", '\t');

In [55]:
gemma_results_strains = readdlm("output/gemma_L_strains.txt", '\t');

In [57]:
# plot(gemma_results_strains[1:1000, :], bulklmm_results_strains_null[1:1000, :])

In [70]:
bulklmm_results_ind_alt_grid

7321×1000 Matrix{Float64}:
 0.144266    0.00396406  0.109569    …  0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569    …  0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569    …  0.0438948   0.567834    0.0483595
 0.144266    0.00396406  0.109569       0.0438948   0.567834    0.0483595
 0.0397299   0.00032194  0.289912       0.352193    0.00037901  0.0156391
 ⋮         